In [50]:
sample_submission_path = "/kaggle/input/playground-series-s5e7/sample_submission.csv"
train_path = "/kaggle/input/playground-series-s5e7/train.csv"
test_path = "/kaggle/input/playground-series-s5e7/test.csv"

In [51]:
sample_submission_path = "/home/yanncauchepin/Datasets/SupervisedLearning/kaggle_introvertsextroverts/sample_submission.csv"
train_path = "/home/yanncauchepin/Datasets/SupervisedLearning/kaggle_introvertsextroverts/train.csv"
test_path = "/home/yanncauchepin/Datasets/SupervisedLearning/kaggle_introvertsextroverts/test.csv"

In [52]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder


In [72]:
import pandas as pd

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_submission_df = pd.read_csv(sample_submission_path)

In [54]:
train_df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [55]:
train_df.isnull().sum()

id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [56]:
test_df.isnull().sum()

id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64

In [76]:
pd.api.types.is_numeric_dtype(train_df["Personality"])

False

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

def impute_nan_non_numeric(df):
    for col in df.columns:
        if df[col].isnull().any():
            if not pd.api.types.is_numeric_dtype(df[col]):
                   # For non-numeric columns, use mode imputation
                df[col] = df[col].fillna(df[col].mode()[0])
                
def impute_nan_numeric(df):
    for col in df.columns:
        if df[col].isnull().any():
            if pd.api.types.is_numeric_dtype(df[col]):
                print(f"Imputing missing values in column: {col}")
                missing_mask = df[col].isnull()
                df_complete = df[~missing_mask]
                df_incomplete = df[missing_mask]
                if df_complete.empty:
                    imputer = SimpleImputer(strategy='mean')
                    df[col] = imputer.fit_transform(df[[col]])
                    continue
                X_complete = df_complete.drop(col, axis=1, errors='ignore')  # Drop target, handle missing columns
                y_complete = df_complete[col]
                X_incomplete = df_incomplete.drop(col, axis=1, errors='ignore') # Drop target, handle missing columns

                # Handle missing values in features using SimpleImputer (before model training)
                # num_imputer = SimpleImputer(strategy='median')  # Or use 'mean', 'most_frequent', 'constant'
                # X_complete = num_imputer.fit_transform(X_complete)
                # X_incomplete = num_imputer.transform(X_incomplete)

                # Train the RandomForestRegressor model
                print(f"{X_incomplete} - {X_complete}")
                model = RandomForestRegressor(n_estimators=100, random_state=42)  # Adjust hyperparameters as needed
                model.fit(X_complete, y_complete)

                # Predict the missing values
                predicted_values = model.predict(X_incomplete)

                # Fill the missing values in the original DataFrame
                df.loc[missing_mask, col] = predicted_values
    return df

In [58]:
train_df.describe()

,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,18524.000000,17334.000000,17344.000000,17058.000000,17470.000000,17260.000000
mean,9261.500000,3.137764,5.265106,4.044319,7.996737,4.982097
std,5347.562529,3.003786,2.753359,2.062580,4.223484,2.879139
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4630.750000,1.000000,3.000000,3.000000,5.000000,3.000000
50%,9261.500000,2.000000,5.000000,4.000000,8.000000,5.000000
75%,13892.250000,4.000000,8.000000,6.000000,12.000000,7.000000
max,18523.000000,11.000000,10.000000,7.000000,15.000000,10.000000


In [59]:
test_df.describe()

,id,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,6175.000000,5750.000000,5778.000000,5709.000000,5825.000000,5767.000000
mean,21611.000000,3.116870,5.287989,4.037835,8.008412,5.028958
std,1782.713288,2.985658,2.758052,2.045207,4.192701,2.867285
min,18524.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20067.500000,1.000000,3.000000,3.000000,5.000000,3.000000
50%,21611.000000,2.000000,5.000000,4.000000,8.000000,5.000000
75%,23154.500000,4.000000,8.000000,6.000000,12.000000,7.000000
max,24698.000000,11.000000,10.000000,7.000000,15.000000,10.000000


In [60]:
train_df.dtypes

id                             int64
Time_spent_Alone             float64
Stage_fear                    object
Social_event_attendance      float64
Going_outside                float64
Drained_after_socializing     object
Friends_circle_size          float64
Post_frequency               float64
Personality                   object
dtype: object

In [61]:
test_df.dtypes

id                             int64
Time_spent_Alone             float64
Stage_fear                    object
Social_event_attendance      float64
Going_outside                float64
Drained_after_socializing     object
Friends_circle_size          float64
Post_frequency               float64
dtype: object

In [97]:
def preprocess_data(df, label_encoders=None):
    
    df = impute_nan(df)
    
    if label_encoders is None:
        label_encoders = {}
    
    for col in df.select_dtypes(include=['object']).columns:
        if col in label_encoders:
            le = label_encoders[col]
            df[col] = le.transform(df[col])
        else:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            label_encoders[col] = le
    
    return df, label_encoders

In [102]:
preprocessed_train_df, label_encoders = preprocess_data(train_df)
preprocessed_test_df, _ = preprocess_data(test_df, label_encoders)

Imputing missing values in column: Time_spent_Alone
          id Stage_fear  Social_event_attendance  Going_outside  \
15        15         No                      7.0            6.0   
29        29        Yes                      1.0            0.0   
31        31         No                      4.0            NaN   
41        41         No                      6.0            5.0   
62        62         No                      6.0            6.0   
...      ...        ...                      ...            ...   
18502  18502         No                      4.0            5.0   
18510  18510         No                      NaN            5.0   
18515  18515         No                      9.0            4.0   
18516  18516         No                      6.0            5.0   
18522  18522        Yes                      1.0            0.0   

      Drained_after_socializing  Friends_circle_size  Post_frequency  \
15                           No                  5.0             8.0   

ValueError: could not convert string to float: 'No'

In [95]:
train_df["Time_spent_Alone"].value_counts()

Time_spent_Alone
0.0     3139
3.0     3081
2.0     3039
1.0     2973
4.0     1079
5.0      633
10.0     587
8.0      582
7.0      581
6.0      574
9.0      574
11.0     492
Name: count, dtype: int64

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.0):
        super(MLP, self).__init__()
        self.layers = nn.Sequential()

        # Input layer
        self.layers.add_module('fc1', nn.Linear(input_size, hidden_size[0]))
        self.layers.add_module('relu1', nn.ReLU())
        if dropout_rate > 0:
            self.layers.add_module('dropout1', nn.Dropout(dropout_rate))

        # Hidden layers
        for i in range(len(hidden_size) - 1):
            self.layers.add_module(f'fc{i+2}', nn.Linear(hidden_size[i], hidden_size[i + 1]))
            self.layers.add_module(f'relu{i+2}', nn.ReLU())
            if dropout_rate > 0:
                self.layers.add_module(f'dropout{i+2}', nn.Dropout(dropout_rate))

        # Output layer
        self.layers.add_module('output', nn.Linear(hidden_size[-1], output_size))
        

    def forward(self, x):
        return self.layers(x)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

input_size = preprocessed_train_df.shape[1] - 1 
output_size = 1  
hidden_size = [32, 64, 64, 32] 
dropout_rate = 0.2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_size, hidden_size, output_size, dropout_rate).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X = preprocessed_train_df.drop(columns=['Personality']).values
Y = preprocessed_train_df['Personality'].values
X_test = preprocessed_test_df.values

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).to(device).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32).to(device).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

num_epochs = 200
batch_size = 32
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
patience = 15  # Number of epochs to wait for improvement
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')
        
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor.to(device)) # Ensure validation data is on the correct device
        val_loss = criterion(val_outputs, Y_val_tensor) # Calculate validation loss
        val_predictions = torch.sigmoid(val_outputs).cpu().numpy()
        val_predictions = (val_predictions > 0.5).astype(int)
        accuracy = accuracy_score(Y_val_tensor.cpu().numpy(), val_predictions) # Ensure Y_val is on CPU and is a numpy array
        print(f'Validation Accuracy: {accuracy:.4f}, Validation Loss: {val_loss.item():.4f}')

        # Early stopping check
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            epochs_no_improve = 0
            # Save the best model
            torch.save(model.state_dict(), 'mlp_best_model.pth')  # Save the model weights
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f'Early stopping triggered after {epoch + 1} epochs!')
                # Load the best model
                model.load_state_dict(torch.load('mlp_best_model.pth'))
                break
    


Validation Accuracy: 0.7355, Validation Loss: 0.6323
Validation Accuracy: 0.7355, Validation Loss: 0.5891
Validation Accuracy: 0.7355, Validation Loss: 0.5932
Validation Accuracy: 0.7355, Validation Loss: 0.5793
Validation Accuracy: 0.7355, Validation Loss: 0.5790
Validation Accuracy: 0.7355, Validation Loss: 0.5779
Validation Accuracy: 0.7355, Validation Loss: 0.5777
Validation Accuracy: 0.7355, Validation Loss: 0.5777
Validation Accuracy: 0.7355, Validation Loss: 0.5777
Epoch [10/200], Loss: 0.5729
Validation Accuracy: 0.7355, Validation Loss: 0.5777
Validation Accuracy: 0.7355, Validation Loss: 0.5770
Validation Accuracy: 0.7355, Validation Loss: 0.5763
Validation Accuracy: 0.7355, Validation Loss: 0.5756
Validation Accuracy: 0.7355, Validation Loss: 0.5761
Validation Accuracy: 0.7355, Validation Loss: 0.5755
Validation Accuracy: 0.7355, Validation Loss: 0.5739
Validation Accuracy: 0.7355, Validation Loss: 0.5747
Validation Accuracy: 0.7355, Validation Loss: 0.5736
Validation Accura

In [ ]:
sample_submission_df

,id,Personality
0,18524,Extrovert
1,18525,Extrovert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Extrovert
...,...,...
6170,24694,Extrovert
6171,24695,Extrovert
6172,24696,Extrovert
6173,24697,Extrovert


In [35]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_predictions = torch.sigmoid(test_outputs).cpu().numpy()
    test_predictions = (test_predictions > 0.5).astype(int)

test_predictions = label_encoders['Personality'].inverse_transform(test_predictions.flatten())

submission_df = pd.DataFrame({
    'id': test_df['id'],
    'Personality': test_predictions.flatten()
})
submission_df.to_csv('submission.csv', index=False)

KeyError: 'Personality'

In [ ]:
submission_df

,Id,Personality
0,18524,Extrovert
1,18525,Introvert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Introvert
...,...,...
6170,24694,Extrovert
6171,24695,Introvert
6172,24696,Extrovert
6173,24697,Extrovert
